In [39]:
import joblib

X_scaler = joblib.load('data/X_scaler.save')
y_scaler = joblib.load('data/y_scaler.save')

In [40]:
import pandas as pd 
expert_agent = pd.read_csv("data_generation_mpc_110_190_6_all.csv")

In [41]:
X = expert_agent[["inflow", "height"]]
y = expert_agent[["speed1_rpm", "speed4_rpm"]]

In [42]:
X_scaled = X_scaler.transform(X)
y_scaled = y_scaler.transform(y)

In [43]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim

class LitNeuralNet(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 50)
        self.fc2 = nn.Linear(50, 10)
        self.fc3 = nn.Linear(10, 2)
        self.train_loss = []
        self.val_loss = []

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        train_loss = nn.functional.mse_loss(y_hat, y)
        self.train_loss.append(train_loss.cpu().item())
        self.log('train_loss', train_loss, on_epoch=True, prog_bar=True)
        return train_loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        val_loss = nn.functional.mse_loss(y_hat, y)
        self.val_loss.append(val_loss.cpu().item())
        self.log('val_loss', val_loss, on_epoch=True, prog_bar=True)
        return val_loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        test_loss = nn.functional.mse_loss(y_hat, y)
        self.log('test_loss', test_loss, on_epoch=True)
        return test_loss

    def configure_optimizers(self):
        # Add L2 regularization with weight_decay
        optimizer = optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-5)
        return optimizer

In [44]:
model = LitNeuralNet()

In [47]:
model = model.load_from_checkpoint("lightning_logs/o5x68pzt/checkpoints/epoch=2-step=4335.ckpt")

In [50]:
from torch.utils.data import DataLoader, TensorDataset
import torch


torch_dataset = torch.tensor(X_scaled, dtype=torch.float32).to('cuda:0')

In [51]:
model.eval()

with torch.no_grad():
    # Assuming you have a tensor 'input_data'
    predictions = model(torch_dataset)


In [54]:
y_scaler.inverse_transform(predictions.cpu())

array([[ 555.94579352,  450.30829197],
       [ 570.86483784,  468.66347745],
       [ 585.97422206,  487.19301043],
       ...,
       [1503.80377783, 1491.90432656],
       [1502.06403059, 1499.81470368],
       [1499.86062344, 1507.24077687]])